In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 大文件摘要文本

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 上執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Vertex AI 工作台開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Thu Ya Kyaw](https://github.com/iamthuya) |


## 概觀

文字摘要是建立文本文件較短版本，同時仍然保留最重要資訊的過程。這對於各種用途很有幫助，例如快速瀏覽長篇文件、取得文章要點或與其他人分享摘要。

儘管摘要簡短段落並非難事，但如果你想摘要大型文件 (例如多頁面 PDF 檔)，仍有幾個挑戰需要克服。在本筆記中，你將逐步了解如何使用生成模型來摘要大型文件。


### 目標

在本教學課程中，你將會透過以下範例了解如何使用生成式模型來整理文字中的資訊：

- 內容填塞方法
- MapReduce 方法
- 包含重疊區段的 MapReduce 方法
- 包含滾動摘要的 MapReduce 方法


### 費用

本教學課程使用 Google Cloud 可計費的組成部分：
- Vertex AI Generative AI Studio

進一步了解 [Vertex AI 價格](https://cloud.google.com/vertex-ai/pricing)， [生成式 AI 價格](https://cloud.google.com/vertex-ai/pricing#generative_ai_models)，以及使用 [價格計算器](https://cloud.google.com/products/calculator/)，根據你預估的使用量，來產生費用估算。


## 開始使用


### 安裝 Vertex AI SDK、其他套件及其依賴項


In [ ]:
!pip install google-cloud-aiplatform PyPDF2 ratelimit backoff --upgrade --quiet --user

**Colab 專屬** : 取消以下區塊註解以重新啟動核心。對於 Vertex AI Workbench，可以使用頂部的按鈕重新啟動終端機。


In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 驗證筆記本環境
* 如果你使用 **Colab** 執行此筆記本，取消註解下方的Cell並繼續。
* 如果你使用 **Vertex AI 工作台** ，請查看[此處](https://github.com/doggy8088/generative-ai/tree/main/setup-env)的設定說明。


In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### 匯入函式庫


**Colab 專用：** 取消下一個Cell註解，以初始化 Vertex AI SDK。對於 Vertex AI Workbench，你不需要執行這項作業。


In [ ]:
# import vertexai

# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import re
import urllib
import warnings
from pathlib import Path

import backoff
import pandas as pd
import PyPDF2
import ratelimit
from google.api_core import exceptions
from tqdm import tqdm
from vertexai.language_models import TextGenerationModel

warnings.filterwarnings("ignore")

### 匯入模型

在這裡，你會載入預先訓練，稱為 `text-bison@001` 的文字生成模型。


In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

### 準備數據文件

開始之前，你需要為以下摘要任務下載 PDF 檔案。


In [ ]:
# Define a folder to store the files
data_folder = "data"
Path(data_folder).mkdir(parents=True, exist_ok=True)

# Define a pdf link to download and place to store the download file
pdf_url = "https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf"
pdf_file = Path(data_folder, pdf_url.split("/")[-1])

# Download the file using `urllib` library
urllib.request.urlretrieve(pdf_url, pdf_file)

在這裡你會看到下載的 pdf 檔案的數頁


In [ ]:
# Read the PDF file and create a list of pages
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# Print three pages from the pdf
for i in range(3):
    text = pages[i].extract_text().strip()
    print(f"Page {i}: {text} \n\n")

## 方法 1：塞入

將資料傳遞到語言模型最簡單的方式，就是將所有資料「塞入」提示中作為脈絡。換句話說，就是簡單地將所有相關資訊包含在提示中，依據你希望模型處理它的順序排列。

在這裡，你將從 pdf 檔案的所有頁面中擷取文字。


In [ ]:
# Read the PDF file and create a list of pages
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# Entry string to concatenate all the extacted texts
concatenated_text = ""

# Loop through the pages
for page in tqdm(pages):
    # Extract the text from the page and remove any leading or trailing whitespace
    text = page.extract_text().strip()

    # Concate the extracted text to the concatenated text
    concatenated_text += text

print(f"There are {len(concatenated_text)} characters in the pdf")

現在，你將建立可在稍後的筆記本中使用的提示範本


In [ ]:
prompt_template = """
    Write a concise summary of the following text delimited by triple backquotes.
    Return your response in bullet points which covers the key points of the text.

    ```{text}```

    BULLET POINT SUMMARY:
"""

在這裡，你將透過 API 使用 LLM 來摘要已萃取的文字。請注意，LLM 目前有輸入文字限制，填入過長的輸入文字可能會不被接受。你可以在此處 [這邊](https://cloud.google.com/vertex-ai/docs/quotas) 閱讀更多關於配額和限制的資訊。

以下程式碼將導致 **異常** ！


In [ ]:
# Define the prompt using the prompt template
prompt = prompt_template.format(text=concatenated_text)

# Use the model to summarize the text using the prompt
summary = generation_model.predict(prompt=prompt, max_output_tokens=1024).text

print(summary)

#### 重試

這個模型以錯誤訊息回應：**400 請求包含無效的參數** ，因為萃取出來的文字太長，生成模型無法處理。

為避免這個問題，你只能輸入一小部分萃取出來的文字 (例如前 30000 個字)。


In [ ]:
# Define the prompt using the prompt template
prompt = prompt_template.format(text=concatenated_text[:30000])

# Use the model to summarize the text using the prompt
summary = generation_model.predict(prompt=prompt, max_output_tokens=1024).text

print(summary)

### 召回

儘管全文對模型而言過大，但你設法使用模型建立包含來自 PDF 部分內容最重要的資訊的簡潔項目清單。因此，以下是使用填充方法的優缺點：

**優點：** 
- 只需要呼叫模型一次。
- 總結文字時，模型可以一次存取所有資料，因此結果可能會更好。

**缺點：** 
- 大多數的模型都有脈絡長度，對於大型文件 (或許多文件)，這無法執行，原因是在於它會導致提示長度大於脈絡長度。
- 這方法只能在較小的資料片段上執行，並且大多數時候不適用於大型文件。

在下一個工作階段，你將探索方法，旨在協助處理超過 LLM 的脈絡長度限制的較長文字。


### 為模型呼叫加入速率限制

當你使用 MapReduce 或其他類似的方法時，你會在短時間內對模型進行多個 API 呼叫。對於你每分鐘可以進行的 API 呼叫數量有設定限制，因此你需要在你的程式碼中加入安全措施，以防止超過限制。這有助於確保你的程式碼順利執行，且不會遇到任何錯誤。

對於這個方法，以下是你將執行的幾個具體事項：
1. 你會使用稱為 [ratelimit](https://pypi.org/project/ratelimit/) 的 Python 函式庫來限制每分鐘的 API 呼叫數量
2. 你會使用稱為 [backoff](https://pypi.org/project/backoff/) 的 Python 函式庫來重試，直到達到最大時間限制

下列函式會透過將呼叫數量限制在**每分鐘 20 個** ，來改進 API 呼叫程序。它也會在遇到**資源耗盡** 例外情況後，暫停並重試呼叫 API。等待時間**會以指數方式增加，直到 5 分鐘為止** ，然後函式將放棄重試。


In [ ]:
CALL_LIMIT = 20  # Number of calls to allow within a period
ONE_MINUTE = 60  # One minute in seconds
FIVE_MINUTE = 5 * ONE_MINUTE


# A function to print a message when the function is retrying
def backoff_hdlr(details):
    print(
        "Backing off {} seconds after {} tries".format(
            details["wait"], details["tries"]
        )
    )


@backoff.on_exception(  # Retry with exponential backoff strategy when exceptions occur
    backoff.expo,
    (
        exceptions.ResourceExhausted,
        ratelimit.RateLimitException,
    ),  # Exceptions to retry on
    max_time=FIVE_MINUTE,
    on_backoff=backoff_hdlr,  # Function to call when retrying
)
@ratelimit.limits(  # Limit the number of calls to the model per minute
    calls=CALL_LIMIT, period=ONE_MINUTE
)

# This function will call the `generation_model.predict` function, but it will retry if defined exceptions occur.
def model_with_limit_and_backoff(**kwargs):
    return generation_model.predict(**kwargs)

## 方法 2：MapReduce

此方法透過首先將大型資料分割成塊狀，然後在每個文本塊上執行提示來執行。對於摘要任務，初始提示的輸出將是該塊狀的摘要。一旦產生了所有初始輸出，便會執行不同的提示以組合它們。

此方法比第一種方法有點複雜，但對於大型數據集更有效。在這裡，你將準備兩個提示範本：一個用於初始摘要步驟，另一個用於最終組合步驟。你將在這個筆記本中後續使用這兩個範本。


In [ ]:
initial_prompt_template = """
    Write a concise summary of the following text delimited by triple backquotes.

    ```{text}```

    CONCISE SUMMARY:
"""

final_prompt_template = """
    Write a concise summary of the following text delimited by triple backquotes.
    Return your response in bullet points which covers the key points of the text.

    ```{text}```

    BULLET POINT SUMMARY:
"""

#### 地圖步驟

在本節中，你將再次閱讀 PDF 文件，並使用模型使用初始提示範本分別總結每個頁面。


In [ ]:
# Read the PDF file and create a list of pages
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# Create an empty list to store the summaries
initial_summary = []

# Iterate over the pages and generate a summary for each page
for page in tqdm(pages):
    # Extract the text from the page and remove any leading or trailing whitespace
    text = page.extract_text().strip()

    # Create a prompt for the model using the extracted text and a prompt template
    prompt = initial_prompt_template.format(text=text)

    # Generate a summary using the model and the prompt
    summary = model_with_limit_and_backoff(prompt=prompt, max_output_tokens=1024).text

    # Append the summary to the list of summaries
    initial_summary.append(summary)

看看從初始 Map 詞組中前幾個摘要。


In [ ]:
print("\n\n".join(initial_summary[:10]))

在這裡，你將會計算初始摘要中的字元數，看看是否夠少可以放入一個提示中。


In [ ]:
len("\n".join(initial_summary))

由於你先前設法在提示中輸入 30,000 個字元，你也可以直接在提示中輸入這個摘要，摘要較少字元。你將在下一步執行此操作。


#### 減少步驟

在這裡，你將建立一個簡化功能，該功能連接來自於初始摘要步驟 (對應步驟) 的摘要，並使用最終提示範本，再次對摘要進行摘要。


In [ ]:
# Define a function to create a summary of the summaries
def reduce(initial_summary, prompt_template):
    # Concatenate the summaries from the inital step
    concat_summary = "\n".join(initial_summary)

    # Create a prompt for the model using the concatenated text and a prompt template
    prompt = prompt_template.format(text=concat_summary)

    # Generate a summary using the model and the prompt
    summary = model_with_limit_and_backoff(prompt=prompt, max_output_tokens=1024).text

    return summary

你可以開始進入下一個步驟，使用最後提示模板和你之前建立的函式，將所有摘要結合起來生成更精簡的摘要。


In [ ]:
# Use defined `reduce` function to summarize the summaries
summary = reduce(initial_summary, final_prompt_template)

print(summary)

#### 召回

你剛剛使用 MapReduce 方法將整篇論文摘要成幾個要點。以下是使用此類方法的優缺點：

**優點：** 
- 可以摘要大量文件
- 可以好好地與平行處理搭配運作，因為摘要頁面的處理彼此獨立

**缺點：** 
- 需要多次呼叫模型
- 由於各個頁面個別摘要，頁面之間的上下文可能會消失


在下一部分，你將嘗試另一種方法，在提示中每次使用多個區塊 (頁面) 進行摘要。


## 方法 3：使用重疊區塊的 MapReduce

這與 MapReduce 類似，但主要差別在於：重疊區塊。這表示會將幾頁摘要在一起，而不是將每頁單獨摘要。這有助於保留區塊之間更多內容或資訊，進而提升結果準確度。

需注意，合併區塊有時候會超過模型設定的 Token 數量限制。如果發生這種情形，你可以實作分塊方法示範或靈活解決此問題 (例如：移除幾個初始區塊)。


#### 繪製步驟

在這個步驟中，你將再次閱讀此 PDF 文件，並使用模型，使用你較早定義的初始提示範本，將**幾頁** 文件摘要在一起。


In [ ]:
# Read the PDF file and create a list of pages
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# Create an empty list to store the extracted text from the pages
text_from_pages = []

# Iterate over the pages and generate a summary for each page
for page in tqdm(pages):
    # Extract the text from the page and remove any leading or trailing whitespace
    text = page.extract_text().strip()

    # Append the extracted text to the list of extracted text
    text_from_pages.append(text)

在這裡你將定義區塊大小 (在此範例中結合的頁面數目) 和摘要區塊。


In [ ]:
CHUNK_SIZE = 2  # number of overlapping pages

# Read the PDF file and create a list of pages
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# Create an empty list to store the summaries
initial_summary = []

# Iterate over the pages and generate a summary for a few pages as one chunk based on `CHUNK_SIZE`
for i in tqdm(range(len(pages))):
    # Select a list of pages to merge as one chunk
    pages_to_merge = [x for x in range(i, i + CHUNK_SIZE) if x < len(pages)]

    extracted_texts = [text_from_pages[x] for x in pages_to_merge]

    # Concatenate the
    text = "\n".join(extracted_texts)

    # Create a prompt for the model using the concatenated text and a prompt template
    prompt = initial_prompt_template.format(text=text)

    # Generate a summary using the model and the prompt
    summary = model_with_limit_and_backoff(prompt=prompt, max_output_tokens=1024).text

    # Append the summary to the list of summaries
    initial_summary.append(summary)

    # If the last page is reached, break the loop
    if pages_to_merge[-1] == len(reader.pages):
        break

看看從初始 Map 詞組中前幾個摘要。


In [ ]:
print("\n\n".join(initial_summary[:10]))

#### 減少步驟

你已準備好繼續執行下一步，將所有摘要結合到一個更小的摘要中，使用最終提示範本和先前提出的函式。


In [ ]:
# Use defined `reduce` function to summarize the summaries
summary = reduce(initial_summary, final_prompt_template)

print(summary)

#### 回顧

該模型可以使用 MapReduce with Overlapping Chunks 方法將整篇論文總結成幾個條列重點。以下列出使用該方法的優缺點：

**優點：** 
- 可以總結大型文件
- 由於連續的頁面一起進行摘要，因此頁面之間的上下文得以保留
- 由於結果彼此獨立，因此可以使用並行處理

**缺點：** 
- 需要多次呼叫模型
- 略慢於純 MapReduce 方法
- 建立更大的輸入文本

在下一個區段，你將嘗試使用另一種方式，利用前一頁的摘要，而非使用全文。


## 方法 4：使用滾動摘要的 MapReduce (精緻) 

在某些情況下，將幾頁合併起來可能太大而無法摘要。為了解決這個問題，現在你將使用一個不同的方法，該方法使用前一階段的初始摘要和下一頁來摘要每個提示。這有助於確保摘要完整且準確，因為它考慮了前一頁的背景。


In [ ]:
initial_prompt_template = """
    Taking the following context delimited by triple backquotes into consideration:

    ```{context}```

    Write a concise summary of the following text delimited by triple backquotes.

    ```{text}```

    CONCISE SUMMARY:
"""

In [ ]:
# Read the PDF file and create a list of pages.
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# Create an empty list to store the summaries.
initial_summary = []

# Iterate over the pages and generate a summary
for idx, page in enumerate(tqdm(pages)):
    # Extract the text from the page and remove any leading or trailing whitespace.
    text = page.extract_text().strip()

    if idx == 0:  # if current page is the first page, no previous context
        prompt = initial_prompt_template.format(context="", text=text)

    else:  # if current page is not the first page, previous context is the summary of the previous page
        prompt = initial_prompt_template.format(
            context=initial_summary[idx - 1], text=text
        )

    # Generate a summary using the model and the prompt
    summary = model_with_limit_and_backoff(prompt=prompt, max_output_tokens=1024).text

    # Append the summary to the list of summaries
    initial_summary.append(summary)

在這裡，你會從初始摘要清單中列出幾條記錄。


In [ ]:
initial_summary[:10]

預計清單中會有一些重複的項目，因為你正在將先前頁面的內容傳送到下一頁。你可以使用 set函式輕鬆移除這些重複項。


In [ ]:
initial_summary = set(initial_summary)  # set() function removes duplicate items

#### 縮減步驟
現在你可以繼續進行下一步，將所有摘要結合到一個更小的摘要中，使用最後的 提示範本 和你之前建立的函式。


In [ ]:
# Use defined `reduce` function to summarize the summaries
summary = reduce(initial_summary, final_prompt_template)

print(summary)

#### 摘要

該模型能使用具滾動摘要的 MapReduce 方法將整份論文總結成幾個要點。以下是使用這種方法的優缺點：

**優點：** 
- 可總結大型文件
- 由於使用前幾頁的內容總結順序頁面，頁面之間的脈絡得以保留

**缺點：** 
- 需要呼叫模型多次
- 由於總結頁面的程序彼此依賴，無法與並行處理搭配順利使用


## 結論

你成功地總結了一份冗長的文件，即使最開始由於輸入提示的限制，這是不可能的。你還了解了總結冗長文件的幾種方法，以及其優缺點。

總結一份冗長的文件可能具有挑戰性。這要求你找出文件的主要觀點，綜合這些資訊，並以一種簡潔連貫的方式展示它們。如果文件很複雜或技術性，這會特別困難。此外，總結一份冗長的文件可能很耗時，因為你需要仔細閱讀並分析文本來確保摘要的準確性和完整性。

儘管這些方法允許你與 LLM 互動並靈活地總結冗長的文件，但你有時可能想透過使用引導或預建方法來加速這個過程。這就是 LangChain 等函式庫發揮作用的地方。你可以在 [此處](https://python.langchain.com/en/latest/modules/models/llms/integrations/google_vertex_ai_palm.html) 閲讀有關頂點 AI 上 LangChain 支援的更多資訊。
